<a href="https://colab.research.google.com/github/dlckdtn62/BIO-PREDICTION/blob/master/RFE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
path = '/content/drive/My Drive/bio'

In [0]:
import pandas as pd
import numpy as np
import math

In [0]:
test = pd.read_csv(path+'/test.csv')
train = pd.read_csv(path+'/train.csv')

In [0]:
trn_dst_filled = train.filter(regex='dst$').replace(0, np.NaN).T.interpolate(method='linear')[::-1].interpolate(method='linear')[::-1].T
tst_dst_filled = test.filter(regex='dst$').replace(0, np.NaN).T.interpolate(method='linear')[::-1].interpolate(method='linear')[::-1].T

train.update(trn_dst_filled)
test.update(tst_dst_filled)

In [0]:
import lightgbm as lgbm
import xgboost as xgb                       # XGBoost 패키지
from sklearn.model_selection import KFold   # K-Fold CV

import warnings
warnings.filterwarnings(action='ignore')

In [0]:
def train_lgbm_model(x_data, y_data, k=5):
    models = []

    kfold = KFold(n_splits=k, shuffle=True, random_state=10)

    for trn_idx, val_idx in kfold.split(x_data):
        x_trn, y_trn = x_data[trn_idx], y_data[trn_idx]
        x_val, y_val = x_data[val_idx], y_data[val_idx]

        trainData = lgbm.Dataset(x_trn, y_trn)
        testData = lgbm.Dataset(x_val, y_val)

        lgb_param = {
            'objective': 'regression', 
            # 'boosting_type': 'dart',
            # 'boosting'  : 'dart',
            'n_estimators': 1000,
            'drop_rate': 0.8, 
            'skip_drop': 0.8, 
            'silent' : False,
            'learning_rate' : 0.05,
            'max_depth' : 12,
            'random_state' : 42,
            'n_jobs' : -1,
            'metric' : 'l1',
            'colsample_bytree' : 0.7,
            'subsample' : 0.7,
            }
        model = lgbm.train(lgb_param, train_set=trainData, num_boost_round=3000, 
                           valid_sets = testData, 
                           verbose_eval=100,
                           early_stopping_rounds=500 # Dart earlystopping 안됨
                           )
        models.append(model)
    return models

In [0]:
# 각 target column별 model 4개 생성 hhb hhb? na ca
models = {}

for label in ['hbo2', 'hhb', 'ca', 'na']:
    print("train column : ", label)
    models[label] = train_lgbm_model(np.array(train.drop(columns = ['id', 'hbo2', 'hhb', 'ca', 'na'])), train[label])

train column :  hbo2
Training until validation scores don't improve for 500 rounds.
[100]	valid_0's l1: 0.682605
[200]	valid_0's l1: 0.662222
[300]	valid_0's l1: 0.651314
[400]	valid_0's l1: 0.647462
[500]	valid_0's l1: 0.644175
[600]	valid_0's l1: 0.642618
[700]	valid_0's l1: 0.64043
[800]	valid_0's l1: 0.639889
[900]	valid_0's l1: 0.639003
[1000]	valid_0's l1: 0.638887
Did not meet early stopping. Best iteration is:
[998]	valid_0's l1: 0.638817
Training until validation scores don't improve for 500 rounds.
[100]	valid_0's l1: 0.668073
[200]	valid_0's l1: 0.647752
[300]	valid_0's l1: 0.641397
[400]	valid_0's l1: 0.635889
[500]	valid_0's l1: 0.632873
[600]	valid_0's l1: 0.630502
[700]	valid_0's l1: 0.629152
[800]	valid_0's l1: 0.627304
[900]	valid_0's l1: 0.627725
[1000]	valid_0's l1: 0.627027
Did not meet early stopping. Best iteration is:
[985]	valid_0's l1: 0.626887
Training until validation scores don't improve for 500 rounds.
[100]	valid_0's l1: 0.665112
[200]	valid_0's l1: 0.6461

In [0]:
sample_submission = pd.read_csv(path + '/sample_submission.csv')
for col in models:
    preds = []
    for model in models[col]:
      preds.append(model.predict(np.array(test.drop(columns = ['id']))))

    pred = np.mean(preds, axis=0)

    sample_submission[col] = pred

In [0]:
models

{'ca': <lightgbm.basic.Booster at 0x7fafdd30bd68>,
 'hbo2': <lightgbm.basic.Booster at 0x7fafe7931ef0>,
 'hhb': <lightgbm.basic.Booster at 0x7fafdd30c780>,
 'na': <lightgbm.basic.Booster at 0x7fafdd30c5f8>}

In [0]:
sample_submission.to_csv(path+'/original_interpolate.csv')

In [0]:
for row in range(len(train)):
  start, finish = 37, len(train.columns)-5
  for column in range(37, len(train.columns)-4):
    if train.iloc[row, column] == 0:
      for j in range(column, 36, -1):
        if train.iloc[row, j] != 0 and not math.isnan(train.iloc[row, j]):
          start = j
          break
      for j in range(column+1, len(train.columns)-4):
        if train.iloc[row, j] != 0 and not math.isnan(train.iloc[row, j]):
          finish = j
          break
      value = (train.iloc[row, start]+train.iloc[row, finish])/(finish-start+1)
      for j in range(1, finish-start):
        if math.isnan(train.iloc[row, start+j]):
          continue
        train.iloc[row, start+j] = value*j+train.iloc[row, start]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: RuntimeWarning: divide by zero encountered in double_scalars
  del sys.path[0]


In [0]:
trn_dst_filled = train.filter(regex='dst$').T.interpolate(method='linear')[::-1].interpolate(method='linear')[::-1].T
train.update(trn_dst_filled)

In [0]:
# from mlxtend.feature_selection import SequentialFeatureSelector as sfs

In [0]:
import lightgbm as lgb
from sklearn.model_selection import train_test_split

In [0]:
for_lgb = train.drop(columns = ['id', 'hhb', 'ca', 'na', 'hbo2'])
len(for_lgb.columns)

71

In [0]:
lgb_param = {'objective': 'regression', 
            'boosting_type': 'dart',
            'boosting'  : 'dart',
            'n_estimators': 1000,
            'drop_rate': 0.8, 
            'skip_drop': 0.8, 
            'silent' : False,
            'learning_rate' : 0.1,
            'max_depth' : 12,
            'random_state' : 42,
            'n_jobs' : -1,
            'metric' : 'l1',
            'colsample_bytree' : 0.7,
            'subsample' : 0.7,
            }

In [0]:
from sklearn.feature_selection import RFE
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor

In [0]:
for label in ['hhb', 'ca', 'na', 'hbo2']:
  train_x, test_x = train_test_split(train, test_size = 0.33)
  # train_data = lgb.Dataset(np.array(train_x.drop(columns = ['id', 'hhb', 'ca', 'na', 'hbo2'])), train_x[label])
  # test_data = lgb.Dataset(np.array(test_x.drop(columns = ['id', 'hhb', 'ca', 'na', 'hbo2'])), test_x[label])
  selector = RFE(RandomForestRegressor(), n_features_to_select= 8, step=3, verbose = 1)
  x = selector.fit(np.array(train_x.drop(columns = ['id', 'hhb', 'ca', 'na', 'hbo2'])), train_x[label])
  print(x.n_features_)

Fitting estimator with 71 features.
Fitting estimator with 68 features.
Fitting estimator with 65 features.
Fitting estimator with 62 features.
Fitting estimator with 59 features.
Fitting estimator with 56 features.
Fitting estimator with 53 features.
Fitting estimator with 50 features.
Fitting estimator with 47 features.
Fitting estimator with 44 features.
Fitting estimator with 41 features.
Fitting estimator with 38 features.
Fitting estimator with 35 features.
Fitting estimator with 32 features.
Fitting estimator with 29 features.
Fitting estimator with 26 features.
Fitting estimator with 23 features.
Fitting estimator with 20 features.
Fitting estimator with 17 features.
Fitting estimator with 14 features.
Fitting estimator with 11 features.
8
Fitting estimator with 71 features.
Fitting estimator with 68 features.


KeyboardInterrupt: ignored

In [0]:
model = RandomForestRegressor().fit(x.transform(train_x.drop(columns = ['id', 'hhb', 'ca', 'na', 'hbo2'])), train_x['hhb'])

In [0]:
pred = model.predict(x.transform(test_x.drop(columns = ['id', 'hhb', 'ca', 'na', 'hbo2'])))

In [0]:
[i for idx, i in enumerate(test_x.drop(columns = ['id', 'hhb', 'ca', 'na', 'hbo2']).columns) if x.support_[idx] == True]

['710_src',
 '740_src',
 '810_src',
 '860_src',
 '880_src',
 '910_src',
 '920_src',
 '990_src']